In [ ]:
from scholarly import scholarly
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
import re
import nest_asyncio
import asyncio

In [ ]:
!pip install tiktoken
!pip install scholarly
!pip install autogen-agentchat
!pip install autogen-ext
!pip install openai

In [12]:
GROQ_API_KEY = "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"
Out_Sep_On = "-------------- OUTPUT : STARTS ----------------"
Out_Sep_Off = "-------------- OUTPUT : ENDS ----------------"
LITERATURE_REVIEW_WORD_COUNT = 500
SINGLE_PAPER_SUMMARY_WORD_COUNT = 100

In [13]:
def collect_Research_papers(query , num_papers):
        """
            Fetch papers
        """
        papers = []
        search_results = scholarly.search_pubs(query)

        for i, paper in enumerate(search_results):
            if i >= num_papers:
                break
            papers.append({
                "title": paper["bib"]["title"],
                "summary": paper["bib"].get("abstract", "No summary available"),
                "link": paper.get("pub_url", "No link available")
            })
        return papers

In [14]:
def get_model_llama():
    model_client = OpenAIChatCompletionClient(
        model="llama3-70b-8192",
        base_url="https://api.groq.com/openai/v1",
        api_key=GROQ_API_KEY,
        model_info={
            "vision": False,
            "function_calling": True,
            "json_output": True,
            "family": "llama3",
        },
    )
    return model_client

In [15]:
def draft_extraction(final_msg):
     if final_msg:
        print("Last Message:")
        print(final_msg)
        final_msg_content = final_msg.content
        print("\nLast message content:\n", final_msg_content)

        pattern = rf"OUTPUT : STARTS(.*?)OUTPUT : ENDS"
        match = re.search(pattern, final_msg_content, re.DOTALL)
        return match.group(1).strip() if match else None

In [16]:
def conv_paper_into_text(papers : list) -> str:
        txt = ""
        for idx, paper in enumerate(papers):
            txt += f"{idx + 1} : Title : {paper['title']}\nDescription : {paper['summary']}\nLINK : {paper['link']}\n\n"
        return txt

In [17]:
async def lit_review_creation(literature_review_topic : str, summary_conext : str):

    writer_agent = AssistantAgent(
    name="literature_review_writer",
    description="""
    Agent for writing literature reviews based on the given topic and research papers.
    This agent should be the first to engage when given a new task.
    """,
    system_message=f"""
    You are a literature review writer tasked with drafting a structured literature review.
    Your final output MUST follow the format below and include the separators `{Out_Sep_On}` and `{Out_Sep_Off}`.

    === REQUIRED TEMPLATE ===
    {Out_Sep_On}

    (Write the final draft here.)

    {Out_Sep_Off}
    =========================

    Instructions:
    - Write a formal literature review with {LITERATURE_REVIEW_WORD_COUNT} words.
    - Use the provided research papers to draft the review.
    - Ask for feedback from the editor agent before finalizing the output.
    - If the editor requests modifications, apply the changes and resubmit.
    - Once the editor **approves the review**, provide the final draft using the template.
    - **Do NOT forget the required separators `{Out_Sep_On}` and `{Out_Sep_Off}`.**
    - End the conversation by saying **"TERMINATE"** when the final version is approved.
    """,
    model_client=get_model_llama(),
)



    editor_agent = AssistantAgent(
    name="literature_review_editor",
    description="""
    Agent for editing and refining literature reviews.
    This agent should be the second to engage when given a new task.
    """,
    system_message=f"""
    You are an Editor and an expert researcher. Your job is to **review and refine the literature review** draft.

    Instructions:
    - **Review the draft for clarity, structure, and completeness.**
    - Provide **constructive feedback** if the draft requires improvements.
    - If the draft is complete and meets user requirements, **approve it** and instruct the writer to submit the final version **strictly in the required template**.

    === REQUIRED FINAL TEMPLATE ===
    {Out_Sep_On}

    (Final approved literature review here.)

    {Out_Sep_Off}
    ================================

    - **Ensure the final output follows this template, including separators `{Out_Sep_On}` and `{Out_Sep_Off}`.**
    - If the template is missing, reject the submission and request a correction.
    - Once the final version is approved, instruct the writer to **end the conversation with "TERMINATE"**.
    """,
    model_client=get_model_llama(),
)



    def get_first_task_str():
        return f"""
        write a literature review on {literature_review_topic} in {LITERATURE_REVIEW_WORD_COUNT} words.

        PLEASE PROVIDE THE FINAL OUTPUT IN FOLLOWING TEMPLATE:
        {Out_Sep_On}
        Write the final draft here.
        {Out_Sep_Off}

        Please use the following papers in drafting the final literature review:
        {summary_conext}
        """

    text_mention_termination = TextMentionTermination("TERMINATE")
    max_messages_termination = MaxMessageTermination(max_messages=8)
    termination = text_mention_termination | max_messages_termination

    literature_review_creation_team = RoundRobinGroupChat([writer_agent,editor_agent],termination_condition=termination)

    await literature_review_creation_team.reset()
    task_result = await Console(literature_review_creation_team.run_stream(task=get_first_task_str()))

    messages = task_result.messages if hasattr(task_result, "messages") else []
    final_msg = messages[-1] if messages else None
    final_draft = draft_extraction(final_msg)
    if final_draft == None:
        final_draft = final_msg.content


    revised_final_draft = final_draft
    task_result = None
    while True:
        user_task = input("Enter your feedback (type 'exit' to leave): ")
        if user_task.lower().strip() == "exit":
            print("- No additional changes requested by user, EXISTING....")
            break

        task_str = f"""
        Please make the changes to the draft

        User Change request : {user_task}

        Draft to edit :
        {revised_final_draft}
        """
        task_result = await Console(literature_review_creation_team.run_stream(task=task_str))
        messages = task_result.messages if hasattr(task_result, "messages") else []
        final_msg = messages[-1] if messages else None
        revised_final_draft = draft_extraction(final_msg)
        if revised_final_draft == None:
            revised_final_draft = final_msg.content

    return revised_final_draft

In [18]:
async def ReviewM(topic, titleD : list[str]):
    print("- Fetching papers from Google Scholar")
    fetched_papers = []

    for paper in titleD:
        fetch_pap = collect_Research_papers(paper, 1)
        if len(fetch_pap) > 0:
            fetched_papers.append(fetch_pap[0])
            link = fetch_pap[0]["link"]
            print(f"{' ' * 3}+ {link}")

    paper_summary_str = conv_paper_into_text(fetched_papers)

    literature_review_draft = await lit_review_creation(topic, paper_summary_str)
    assert literature_review_draft is not None

    print("FINAL DRAFT: ")
    print(literature_review_draft)

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

discussion_theme = "Multi-Agent LLM Systems"
titleD = [
    "Self-Adaptive Large Language Model (LLM)-Based Multiagent Systems",
    "Toward Semantic MAC Protocols for 6G: From Protocol Learning to Language-Oriented Approaches",
    "StackRAG Agent: Improving Developer Answers with Retrieval-Augmented Generation",
    "Enhancing Efficiency and Flexibility of Rapid Prototyping for Scalable Multimodal Intelligent Agents",
    "Mitigating the OWASP Top 10 For Large Language Models Applications using Intelligent Agents",
    "Benchmarking AutoGen with different large language models"

]

await ReviewM(discussion_theme,titleD)

- Fetching papers from Google Scholar
   + https://ieeexplore.ieee.org/abstract/document/10336211/
   + https://ieeexplore.ieee.org/abstract/document/10742906/
   + https://ieeexplore.ieee.org/abstract/document/10795043/
   + https://ieeexplore.ieee.org/abstract/document/10771289/
   + https://ieeexplore.ieee.org/abstract/document/10532874/
   + https://ieeexplore.ieee.org/abstract/document/10605443/
---------- user ----------

        write a literature review on Multi-Agent LLM Systems in 500 words.

        PLEASE PROVIDE THE FINAL OUTPUT IN FOLLOWING TEMPLATE:
        -------------- OUTPUT : STARTS ----------------
        Write the final draft here.
        -------------- OUTPUT : ENDS ----------------

        Please use the following papers in drafting the final literature review:
        1 : Title : Self-adaptive large language model (llm)-based multiagent systems
Description : The complexity of managing multiagent systems (MASs) in autonomic computing can be mitigated using a 